In [1]:
from tqdm import tqdm
from itertools import product
from scipy.signal import welch
%run Utility_Functions.ipynb # import all the utility functions
%run parameters.py # read parameters from the text file 

ERROR:root:File `'parameters.py'` not found.


In [14]:
def _eeg_data(subject, trial):
    """
    Get the eeg values of the 

    Parameter
    ----------
    subject : string of subject ID e.g. 7707 
    trial   : trial of which the powed spectral  
    
    Returns
    ----------
    eeg_data : eoched eeg data

    """
    read_path = '../Cleaned Data/' + subject + '/EEG/'
    cleaned_epochs = mne.read_epochs(read_path + subject + '_' + trial + '_' + str(epoch_length) + '_cleaned_epo.fif', verbose=False)
    
    # Convert to micro volts 
    eeg_epoch_data = cleaned_epochs.get_data()*1e6
        
    return eeg_epoch_data


def _one_hot_encode(length, category):
    """
    Get the one hot encoded value

    Parameter
    ----------
    length     : required lenght of the matrix (m x category)
    category   : caterory 
    
    Returns
    ----------
    y : one hot encoded matrix (m x category)

    """
    y = np.zeros((length, len(category)))
    y[:, category.index(1)]=1
    
    return y
    


def all_subject_data(subjects, trials):
    """
    Get the ir_index of all the subjects

    Parameter
    ----------
    subject : string of subject ID e.g. 7707 
    trial   : trial of which the powed spectral  
    
    Returns
    ----------
    overall_data : numpy array containing all the ir_index, signature: [n_subjects, n_trials]

    """
    x = np.empty([0, n_electrodes, 256*epoch_length])
    y = np.empty([0, 2])
    
    # Loop over all subjects and trials
    for subject, trial in tqdm(product(subjects, trials)):
        
        if (trial=='LowFine') or (trial=='HighGross'):
            x_temp = _eeg_data(subject, trial)
            y_temp = _one_hot_encode(len(x_temp), [0,1])
            
        elif trial=='HighFine':
            x_temp = _eeg_data(subject, trial)
            y_temp = _one_hot_encode(len(x_temp), [1,0])
        elif trial=='LowGross':
            x_temp = _eeg_data(subject, trial)
            y_temp = _one_hot_encode(len(x_temp), [1,0])
        
        x_temp = x_temp[:,0:n_electrodes,0:256*epoch_length]
        y = np.concatenate((y, y_temp), axis=0)
        x = np.concatenate((x, x_temp), axis=0)
    
    return x, y


if __name__ == "__main__":
    
    save = True
    multi_class = False
    x, y = all_subject_data(subjects, trials)
    
    if save:
        if multi_class:
            np.save('Y_Multi_Class.npy', y)
            np.save('X_Multi_Class.npy', x)
        else:
            np.save('Y.npy', y)
            np.save('X.npy', x)
            
    print('All done!')

72it [00:05,  7.40it/s]


All done!
